# Projet : Systeme de recommandation d'images

## Presentation

Dans ce projet, vous allez construire un **systeme de recommandation d'images** qui suggere des images aux utilisateurs en fonction de leurs preferences. Ce projet met en pratique toutes les competences acquises lors des seances pratiques : analyse de donnees, visualisation, regroupement, classification et apprentissage automatique.

**Duree** : 3 seances pratiques
**Taille de l'equipe** : 2-3 etudiants
**Livrables** :
1. Un notebook Jupyter (`Nom1_Nom2_[Nom3].ipynb`)
2. Un rapport de synthese de 4 pages (PDF)

---

## Objectifs d'apprentissage

En completant ce projet, vous serez capable de :
- Automatiser la collecte de donnees a partir de sources web
- Extraire et traiter les metadonnees d'images
- Appliquer des algorithmes de regroupement pour analyser les caracteristiques des images
- Construire des profils de preferences utilisateur
- Implementer un algorithme de recommandation
- Visualiser efficacement les donnees
- Ecrire des tests complets pour votre systeme

---

## Architecture du projet

Le systeme est compose de 7 taches interconnectees :

```
┌─────────────────────────────────────────────────────────────────┐
│               SYSTEME DE RECOMMANDATION D'IMAGES                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐       │
│  │ 1. Collecte  │───▶│ 2. Etiquetage│───▶│ 3. Analyse   │       │
│  │ de donnees   │    │ & Annotation │    │ de donnees   │       │
│  └──────────────┘    └──────────────┘    └──────────────┘       │
│        │                    │                   │                │
│        ▼                    ▼                   ▼                │
│  ┌──────────────────────────────────────────────────────┐       │
│  │          Fichiers JSON (Stockage des metadonnees)     │       │
│  └──────────────────────────────────────────────────────┘       │
│        │                    │                   │                │
│        ▼                    ▼                   ▼                │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐       │
│  │ 4. Visuali-  │    │ 5. Systeme de│    │ 6. Tests     │       │
│  │ sation       │    │ recommandat. │    │              │       │
│  └──────────────┘    └──────────────┘    └──────────────┘       │
│                             │                                    │
│                             ▼                                    │
│                    ┌──────────────┐                              │
│                    │ 7. Rapport   │                              │
│                    │ de synthese  │                              │
│                    └──────────────┘                              │
└─────────────────────────────────────────────────────────────────┘
```

![Architecture](../../images/Project-Architecture.png "Architecture")

---

## Projet partie 1

---

### Tache 1 : Collecte de donnees

#### Objectif
Collecter au moins **100 images sous licence ouverte** avec leurs metadonnees.

#### Ce que vous devez faire

1. **Creer une structure de dossiers** :
   ```
   projet/
   ├── images/           # Images telechargees
   ├── data/             # Fichiers de metadonnees JSON
   └── projet.ipynb      # Votre notebook
   ```

2. **Trouver des sources d'images** (une ou plusieurs) :
   - [Wikimedia Commons](https://commons.wikimedia.org/) - Utilisez des requetes SPARQL (comme dans le TP 1)
   - [Unsplash API](https://unsplash.com/developers) - API gratuite pour des images de haute qualite
   - [Pexels API](https://www.pexels.com/api/) - Photos libres de droits
   - [Flickr API](https://www.flickr.com/services/api/) - Images Creative Commons

3. **Telecharger les images par programme** en utilisant les techniques du TP 1, Exercice 6

4. **Extraire et sauvegarder les metadonnees** de chaque image :
   - Nom du fichier image
   - Dimensions de l'image (largeur, hauteur)
   - Format du fichier (.jpg, .png, etc.)
   - Taille du fichier (en Ko)
   - URL source
   - Informations de licence
   - Donnees EXIF (si disponibles) : modele d'appareil photo, date de prise de vue, etc.

#### Resultat attendu
- Dossier `images/` contenant 100+ images
- `data/images_metadata.json` contenant les metadonnees de toutes les images

#### Conseils
- Utilisez `PIL` pour obtenir les dimensions de l'image
- Utilisez `os.path.getsize()` pour obtenir la taille du fichier
- Utilisez l'extraction EXIF (voir TP 2, Exercice 2)
- Stockez les metadonnees sous forme de liste de dictionnaires au format JSON

---